# Navigating the Night Sky

In [47]:
import numpy as np
import astropy
from astropy.table import QTable
from astroplan import Observer, FixedTarget
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
import pytz
from astroplan import download_IERS_A
from astroplan import AltitudeConstraint, AirmassConstraint, AtNightConstraint
from astroplan import observability_table
from astroplan import moon_illumination

In [48]:
download_IERS_A()

## Problem 1

All right, let's pull in the data!

In [49]:
loud_galaxies = QTable.read('01_data.csv', format = 'ascii.csv' )
loud_galaxies

iau_name,alt_name,ra,dec,class
str9,str16,str10,str9,str14
0454-8105,PKS 0454-81,04 50 05.0,-81 01 02,QSO RADIO LOUD
0621-7841,PKS 0621-786,06 18 30.3,-78 43 02,QSO RADIO LOUD
0858-7707,PKS 0858-77,08 57 42.7,-77 19 32,QSO RADIO LOUD
0312-7703,PKS 0312-77,03 11 54.7,-76 51 52,QSO RADIO LOUD
0202-7634,PKS 0202-76,02 02 13.0,-76 20 06,QSO RADIO LOUD
0842-7529,PKS 0842-75,08 41 27.5,-75 40 26,QSO RADIO LOUD
0637-7513,PKS 0637-75,06 35 46.5,-75 16 17,QSO RADIO LOUD
2300-6823,PKS 2300-683,23 03 43.6,-68 07 37,QSO RADIO LOUD
2111-6806,1E 2111-681,21 15 44.1,-67 54 04,QSO RADIO LOUD


Beautiful. Now, we're going to set our observation location at Kitt Peak so we can later scoop out the galaxies we can actually see.

In [50]:
kitt = Observer.at_site('Kitt Peak')

Here's the fun part. We need to define when we're observing, add constraints so that we don't get crummy data, and then skim the table to find the galaxies which fit our requirements.

In [78]:
l = len(loud_galaxies)

targs = []

for i in range(l):
    targs.append(FixedTarget(coord = SkyCoord(ra=(loud_galaxies[i]['ra']), dec=(loud_galaxies[i]['dec']), unit=(u.hourangle, u.deg)), name=loud_galaxies[i]['alt_name']))


time_range = Time(["2018-04-15 00:01", "2018-04-30 23:59"])

cons = [AirmassConstraint(2), AtNightConstraint.twilight_astronomical()]

obs_tab = observability_table(cons, kitt, targs, time_range=time_range)

obs_targs = []

for i in range(len(obs_tab)):
    if obs_tab[i]['ever observable']:
        obs_targs.append([obs_tab[i]['target name'],obs_tab[i]['fraction of time observable']])

obs_targs.sort(key=lambda x: x[1])

def most_obs(n):
    output = "The " + str(n) + " most (or equally) observable Radio Loud Galaxies in the 2nd half of April: \n"
    for i in range(0, n):
        output += str(i+1) + ". " + obs_targs[len(obs_targs)-(1+i)][0] + " is observable for " + str(obs_targs[len(obs_targs)-(1+i)][1]) + " of the second half of April. \n"
    return output


print(most_obs(10))

The 10 most (or equally) observable Radio Loud Galaxies in the 2nd half of April: 
1. S5 1150+81 is observable for 0.3294270833333333 of the second half of April. 
2. 1221+75W1 is observable for 0.3294270833333333 of the second half of April. 
3. S5 1345+73 is observable for 0.3294270833333333 of the second half of April. 
4. S5 1520+72 is observable for 0.3294270833333333 of the second half of April. 
5. 3C 309.1 is observable for 0.3294270833333333 of the second half of April. 
6. 4C 69.18 is observable for 0.3294270833333333 of the second half of April. 
7. 4C 65.15 is observable for 0.3294270833333333 of the second half of April. 
8. S4 1435+63 is observable for 0.3294270833333333 of the second half of April. 
9. MS 14428+6344 is observable for 0.3294270833333333 of the second half of April. 
10. 4C 58.29 is observable for 0.3294270833333333 of the second half of April. 



I love it when code works. Mama mia, that's a spicy meat-a-ball.

## Problem 2

All right, how awful is the moon going to be? We looked at the moon illumination on the first and last day of our observing window to see how the awfulness would evolve through out that time.

In [52]:
time1 = Time("2018-04-15 00:01", format='iso')
moon_illumination(time1)

0.015518176308118248

That's nice.

In [53]:
time2 = Time("2018-04-30 23:59", format='iso')
moon_illumination(time2)

0.9884801802294492

That's not nice.

In [54]:
mrise1 = Time(kitt.moon_rise_time(time1, 'nearest'), format = 'iso')
mset1 = Time(kitt.moon_set_time(time1, 'next'), format = 'iso')
mrise2 = Time(kitt.moon_rise_time(time2, 'nearest'), format = 'iso')
mset2 = Time(kitt.moon_set_time(time2, 'nearest'), format = 'iso')
print(mrise1)
print(mset1)
print(mrise2)
print(mset2)

2018-04-14 12:29:13.299
2018-04-15 00:40:53.630
2018-05-01 03:00:10.395
2018-04-30 13:18:50.720


Well, it seems the best observations will be conducted in the first half of the window seeing as how the moon rears it's ugly head at 3:00am in the latter half of our window.

## Problem 3

First, we'll check if the moon will be around more within our observing hours.

In [55]:
time3 = Time("2018-05-15 00:01", format='iso')
time4 = Time("2018-05-31 23:59", format='iso')

mrise3 = Time(kitt.moon_rise_time(time3, 'nearest'), format = 'iso')
mset3 = Time(kitt.moon_set_time(time3, 'next'), format = 'iso')
mrise4 = Time(kitt.moon_rise_time(time4, 'nearest'), format = 'iso')
mset4 = Time(kitt.moon_set_time(time4, 'nearest'), format = 'iso')
print(mrise3)
print(mset3)
print(mrise4)
print(mset4)

2018-05-14 12:15:25.908
2018-05-15 01:32:03.083
2018-06-01 04:27:46.054
2018-05-31 14:01:25.886


Not too much different from April. Is there any remarkable difference in phase?

In [56]:
moon_illumination(time3)

0.005213023368624947

In [57]:
moon_illumination(time4)

0.9462180748295312

Hey, it's even better in the first half of hour observing window than last month! Let's take a gander at whether or not our top ten are still our top ten...

In [62]:
for i in range(l):
    targs.append(FixedTarget(coord=SkyCoord(ra=(loud_galaxies[i]['ra']), dec=(loud_galaxies[i]['dec']), unit=(u.hourangle, u.deg)), name=loud_galaxies[i]['alt_name']))


time_range2 = Time(["2018-05-15 00:01", "2018-05-31 23:59"])

obs_tab2 = observability_table(cons, kitt, targs, time_range=time_range2)

obs_targs2 = []

for i in range(len(obs_tab2)):
    if obs_tab2[i]['ever observable']:
        obs_targs2.append([obs_tab2[i]['target name'],obs_tab2[i]['fraction of time observable']])

obs_targs2.sort(key=lambda x: x[1])

def most_obs(n):
    output = "The " + str(n) + " most (or equally) observable Radio Loud Galaxies in the 2nd half of May: \n"
    for i in range(0, n):
        output += str(i+1) + ". " + obs_targs2[len(obs_targs2)-(1+i)][0] + " is observable for " + str(obs_targs2[len(obs_targs2)-(1+i)][1]) + " of the second half of April. \n"
    return output


print(most_obs(10))

The 10 most (or equally) observable Radio Loud Galaxies in the 2nd half of May: 
1. S5 1726+76 is observable for 0.2867647058823529 of the second half of April. 
2. S5 1345+73 is observable for 0.2867647058823529 of the second half of April. 
3. S5 1520+72 is observable for 0.2867647058823529 of the second half of April. 
4. 3C 309.1 is observable for 0.2867647058823529 of the second half of April. 
5. 4C +69.21 is observable for 0.2867647058823529 of the second half of April. 
6. 4C 69.18 is observable for 0.2867647058823529 of the second half of April. 
7. S4 1716+68 is observable for 0.2867647058823529 of the second half of April. 
8. EF B1753+6543 is observable for 0.2867647058823529 of the second half of April. 
9. 4C 65.21 is observable for 0.2867647058823529 of the second half of April. 
10. S4 1435+63 is observable for 0.2867647058823529 of the second half of April. 



Yikes. Notice that the amount of time these are observable are down by 4% of the top 10 in April. The ones we're looking for are doing even worse.